# Master Orientation V2

Major improvement over v1 is to do the depth adjustment between the two pairs of tracks, rather than the whole chunk

## Import packages

In [1]:
# general
import numpy as np
import pandas as pd
import math
from scipy import stats

# progress bar
from tqdm import tqdm

# plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# my functions/classes
import sys
sys.path.append("../core_scripts/")
from ECMclass import ECM

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## User Inputs

In [2]:
# paths
path_to_data = '../../data/'
path_to_angles = '../../data/angles/'
metadata_file = 'metadata.csv'
path_to_angles = '../../data/angles'

# smoothing window
window = 10

meta = pd.read_csv(path_to_data+metadata_file)

## Minor Functions

In [3]:
# define function to find unique elements in list
def unique(list1):
 
    # initialize a null list
    unique_list = []
 
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    
    return(unique_list)

## Major Functions

In [10]:

def compute_dip(data):

    column_names = ['section']
    unique_sec = unique(sections)
    df = pd.DataFrame(unique_sec,columns=column_names)
    df['depth'] = 0

    # set up the angle range
    angle_res = 0.1
    angle_low = -75
    angle_high = 75
    test_angle = np.linspace(angle_low,angle_high,int((angle_high-angle_low)/angle_res+1))

    # loop through each file
    for d in data:

        # Print the current file being processed
        print("Calculating File - "+d.core+
              " - "+d.section+
              " - "+d.face+
              " - "+d.ACorDC)
        
        # assign local variables
        depth = d.depth_s
        meas = d.meas_s
        button = d.button_s
        y_list = d.y_s
        y_vec = d.y_vec

        # set empty lists for angles, scores, and lengths
        angles = []
        scores = []
        lengths = []

        # loop through each pair of tracks
        for i in range(len(y_vec)-1):
            for j in range(i+1,len(y_vec)):
                
                # get the y values for the two tracks
                y1 = y_vec[i]
                y2 = y_vec[j]
                ymid = (y1+y2)/2
                y_space = abs(y1-y2)/2/1000

                # Print the current pair of tracks being processed
                print("    Calculating Pair - "+str(i)+" - "+str(j)+" - y1: "+str(y1)+" - y2: "+str(y2))

                # check y1 and y2 are seprated by more than 5mm
                if abs(y1-y2) > 5:

                    # get the depth, meas, and button values for the two tracks
                    depth1 = depth[y_list==y1]
                    depth2 = depth[y_list==y2]
                    meas1 = meas[y_list==y1]
                    meas2 = meas[y_list==y2]
                    button1 = button[y_list==y1]
                    button2 = button[y_list==y2]
                    
                    # get a common depth
                    common_depths = np.arange(np.round(min(depth1.min(),depth2.min()),3),np.round(max(depth1.max(),depth2.max()),3),0.001)

                    # interpolate button and meas values to common depths
                    button1 = np.interp(common_depths, depth1, button1)
                    button2 = np.interp(common_depths, depth2, button2)

                    # find the longest stretch of common depths where button is 0 for both tracks
                    button_com = np.logical_and(button1==0,button2==0)
                    common_depths = common_depths[button_com]

                    # check that button_com is not empty
                    if common_depths.size != 0:
                    
                        # get dmin and dmax
                        dmin = common_depths.min() + np.tan(np.deg2rad(angle_high)) * y_space + 0.001
                        dmax = common_depths.max() - np.tan(np.deg2rad(angle_high)) * y_space - 0.001

                        # if overall length is >40 
                        if dmax - dmin > 0.4:

                            # compute a vector at 1mm spacing between dmin and dmax. Round up from dmin and down from dmax
                            dmin = np.ceil(dmin * 1000) / 1000
                            dmax = np.floor(dmax * 1000) / 1000
                            cds = np.arange(dmin, dmax, 0.001)

                            # save length
                            lengths.append(np.round(dmax - dmin,3))

                            # # update meas1 and meas2 to be at common depths
                            # meas1 = np.interp(cds, common_depths, meas1)
                            # meas2 = np.interp(cds, common_depths, meas2)

                            # print updated dmin and dmax
                            #print("        dmin: "+str(dmin)+" - dmax: "+str(dmax))

                            # now, loop through each angle
                            max_rho = 0
                            best_angle = 0
                            for angle in test_angle:

                                # get the offset for the angle
                                offset = np.tan(np.deg2rad(angle)) * y_space
                                depth1_angle = depth1 + offset
                                depth2_angle = depth2 - offset

                                # # get the meas values for the two tracks where depth1 and depth2 are within dmin and dmax
                                # meas1_angle = meas1[(depth1_angle >= dmin) & (depth1_angle <= dmax)]
                                # meas2_angle = meas2[(depth2_angle >= dmin) & (depth2_angle <= dmax)]

                                # interpolate the meas values to the common depths
                                meas1_angle= np.interp(cds, np.flip(depth1_angle), np.flip(meas1))
                                meas2_angle = np.interp(cds, np.flip(depth2_angle), np.flip(meas2))

                                # compute the correlation between the two tracks
                                rho, pval = stats.pearsonr(meas1_angle, meas2_angle)
                                if rho>max_rho:
                                    max_rho = np.round(rho,5)
                                    best_angle = np.round(angle,5)
                    
                            scores.append(max_rho)
                            angles.append(best_angle)

                        else:
                            print("        Not enough common depths for pair")

                else:
                    print("        Not enough separation between tracks for pair")
                    continue

                # Print the best angle and max rho
                #print("        Best angle: "+str(best_angle)+" - Max rho: "+str(max_rho))


        # add data to dataframe
        sec_idx = unique_sec.index(d.section)
        df.loc[sec_idx,'section'] = d.section
        df.loc[sec_idx,'depth'] = np.mean(d.depth)
        angle_rowname = d.ACorDC + '-'+d.face + '-angles'
        score_rowname = d.ACorDC + '-'+d.face + '-scores'
        length_rowname = d.ACorDC + '-'+d.face + '-length'
        if angle_rowname not in df.columns:
            df[angle_rowname] = None
            df[score_rowname] = None
            df[length_rowname] = None
        df.at[sec_idx, angle_rowname] = angles
        df.at[sec_idx, score_rowname] = scores
        df.at[sec_idx, length_rowname] = lengths


    return df



## Compute ALHIC2302 orientation calculations

In [8]:
# import each script as an ECM class item
data = []
cores = []
sections = []
faces = []
ACorDCs = []
max_tracks = 0
for index,row in meta.iterrows():
    
    core = row['core']
    section = row['section']
    section_num = section.split("_")
    face = row['face']
    ACorDC = row['ACorDC']
    
    if core == 'alhic2302' and ACorDC == 'AC':
        
        print("Reading "+core+", section "+section+'-'+face+'-'+ACorDC)
    
        data_item = ECM(core,section,face,ACorDC)
        data_item.rem_ends(10)
        data_item.smooth(window)
        data.append(data_item)
        
        cores.append(core)
        sections.append(section)
        faces.append(face)
        ACorDCs.append(ACorDC)
        
        if len(data_item.y_vec)>max_tracks:
            max_tracks = len(data_item.y_vec)

Reading alhic2302, section 10-l-AC
Reading alhic2302, section 10-t-AC
Reading alhic2302, section 12-l-AC
Reading alhic2302, section 12-t-AC
Reading alhic2302, section 13-l-AC
Reading alhic2302, section 13-t-AC
Reading alhic2302, section 14-l-AC
Reading alhic2302, section 14-t-AC
Reading alhic2302, section 15-l-AC
Reading alhic2302, section 15-t-AC
Reading alhic2302, section 16-l-AC
Reading alhic2302, section 16-t-AC
Reading alhic2302, section 17-l-AC
Reading alhic2302, section 17-t-AC
Reading alhic2302, section 18-l-AC
Reading alhic2302, section 18-t-AC
Reading alhic2302, section 19-l-AC
Reading alhic2302, section 19-t-AC
Reading alhic2302, section 20-l-AC
Reading alhic2302, section 20-t-AC
Reading alhic2302, section 21-l-AC
Reading alhic2302, section 21-t-AC
Reading alhic2302, section 22-l-AC
Reading alhic2302, section 22-t-AC
Reading alhic2302, section 22-t-AC
Reading alhic2302, section 23-l-AC
Reading alhic2302, section 23-t-AC
Reading alhic2302, section 24-l-AC
Reading alhic2302, s

In [11]:
alhic2302 = compute_dip(data)

# save the dataframe to a csv file
alhic2302.to_pickle('../../data/angles/alhic2302_angles_v2.df')

Calculating File - alhic2302 - 10 - l - AC
    Calculating Pair - 0 - 1 - y1: 61.231 - y2: 81.231
    Calculating Pair - 0 - 2 - y1: 61.231 - y2: 101.231
    Calculating Pair - 0 - 3 - y1: 61.231 - y2: 121.231
    Calculating Pair - 0 - 4 - y1: 61.231 - y2: 141.231
    Calculating Pair - 0 - 5 - y1: 61.231 - y2: 151.478
    Calculating Pair - 1 - 2 - y1: 81.231 - y2: 101.231
    Calculating Pair - 1 - 3 - y1: 81.231 - y2: 121.231
    Calculating Pair - 1 - 4 - y1: 81.231 - y2: 141.231
    Calculating Pair - 1 - 5 - y1: 81.231 - y2: 151.478
    Calculating Pair - 2 - 3 - y1: 101.231 - y2: 121.231
    Calculating Pair - 2 - 4 - y1: 101.231 - y2: 141.231
    Calculating Pair - 2 - 5 - y1: 101.231 - y2: 151.478
    Calculating Pair - 3 - 4 - y1: 121.231 - y2: 141.231
    Calculating Pair - 3 - 5 - y1: 121.231 - y2: 151.478
    Calculating Pair - 4 - 5 - y1: 141.231 - y2: 151.478
Calculating File - alhic2302 - 10 - t - AC
    Calculating Pair - 0 - 1 - y1: 72.482 - y2: 92.482
    Calculatin

## Complete ALHIC2201 Orientation Calculations

In [12]:
# Read in metadata and import data - ALHIC2201

# import each script as an ECM class item
data = []
cores = []
sections = []
faces = []
ACorDCs = []
max_tracks = 0
for index,row in meta.iterrows():
    
    core = row['core']
    section = row['section']
    section_num = section.split("_")
    face = row['face']
    ACorDC = row['ACorDC']
    
    if core == 'alhic2201' and ACorDC == 'AC':
            
        
        print("Reading "+core+", section "+section+'-'+face+'-'+ACorDC)
    
        data_item = ECM(core,section,face,ACorDC)
        data_item.rem_ends(10)
        data_item.smooth(window)
        data.append(data_item)
        
        cores.append(core)
        sections.append(section)
        faces.append(face)
        ACorDCs.append(ACorDC)
        
        if len(data_item.y_vec)>max_tracks:
            max_tracks = len(data_item.y_vec)



Reading alhic2201, section 10_1-r-AC
Reading alhic2201, section 10_1-t-AC
Reading alhic2201, section 11_1-r-AC
Reading alhic2201, section 11_1-t-AC
Reading alhic2201, section 12_1-r-AC
Reading alhic2201, section 12_1-t-AC
Reading alhic2201, section 13_1-r-AC
Reading alhic2201, section 13_1-t-AC
Reading alhic2201, section 14_1-r-AC
Reading alhic2201, section 14_1-t-AC
Reading alhic2201, section 15_1-r-AC
Reading alhic2201, section 15_1-t-AC
Reading alhic2201, section 16_1-r-AC
Reading alhic2201, section 16_1-t-AC
Reading alhic2201, section 18_1-r-AC
Reading alhic2201, section 18_1-t-AC
Reading alhic2201, section 19_1-r-AC
Reading alhic2201, section 19_1-t-AC
Reading alhic2201, section 1_1-r-AC
Reading alhic2201, section 1_1-t-AC
Reading alhic2201, section 20_1-r-AC
Reading alhic2201, section 20_1-t-AC
Reading alhic2201, section 21_2-r-AC
Reading alhic2201, section 21_2-t-AC
Reading alhic2201, section 22_1-r-AC
Reading alhic2201, section 22_1-t-AC
Reading alhic2201, section 23_1-r-AC
Rea

In [13]:
alhic2201 = compute_dip(data)

alhic2201.to_pickle('../../data/angles/alhic2201_angles_v2.df')

Calculating File - alhic2201 - 10_1 - r - AC
    Calculating Pair - 0 - 1 - y1: 81.19 - y2: 93.19
    Calculating Pair - 0 - 2 - y1: 81.19 - y2: 105.19
    Calculating Pair - 0 - 3 - y1: 81.19 - y2: 112.199
    Calculating Pair - 1 - 2 - y1: 93.19 - y2: 105.19
    Calculating Pair - 1 - 3 - y1: 93.19 - y2: 112.199
    Calculating Pair - 2 - 3 - y1: 105.19 - y2: 112.199
Calculating File - alhic2201 - 10_1 - t - AC
    Calculating Pair - 0 - 1 - y1: 34.667 - y2: 52.667
    Calculating Pair - 0 - 2 - y1: 34.667 - y2: 70.667
    Calculating Pair - 0 - 3 - y1: 34.667 - y2: 88.667
    Calculating Pair - 0 - 4 - y1: 34.667 - y2: 106.667
    Calculating Pair - 0 - 5 - y1: 34.667 - y2: 117.411
        Not enough common depths for pair
    Calculating Pair - 1 - 2 - y1: 52.667 - y2: 70.667
    Calculating Pair - 1 - 3 - y1: 52.667 - y2: 88.667
    Calculating Pair - 1 - 4 - y1: 52.667 - y2: 106.667
    Calculating Pair - 1 - 5 - y1: 52.667 - y2: 117.411
    Calculating Pair - 2 - 3 - y1: 70.667 